Nasa Satellite images:

1.	Suppose we want to build a Computer vision dataset that involves satellite images. 

2.	Your tasks are the following:

    a.	Collect satellite images from  https://earthobservatory.nasa.gov/images
    
        ■	Make sure to render the whole page using selenium and then scrape the data.
        
        ■	Create a repo and name it Images, Save the crowled images based on their titles. 
        
        ■	Create a dictionary where the keys are the images/titles and the values are the images’ descriptions.

In [ ]:
import warnings
import requests
import random
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import json

class ImageScraper:
    def __init__(self):
        warnings.filterwarnings("ignore")
        self.driver = webdriver.Chrome(executable_path="chromedriver.exe")
        self.img_dict = {}
        self.img = []
        self.txt = []

    def scrape_images(self, url, num_pages):
        self.driver.get(url)
        self.driver.maximize_window()
        time.sleep(3)

        for _ in range(num_pages):
            try:
                self.driver.find_element(By.XPATH, "//a[@title='explore more']").click()
            except Exception as e:
                print('Exception:', e)
            time.sleep(random.randint(3, 7))

        imgs = self.driver.find_elements(By.CSS_SELECTOR, "div.thumbnail-image")
        
        for i, _ in enumerate(imgs):
            img_add = imgs[i].get_attribute('innerHTML')
#             Looking for Image address
            start_url = img_add.find("img src=")
            end_url = img_add.find(".jpg")
            image_url = img_add[start_url+9:end_url+4]
#             Looking for Image Name
            start_name = img_add.find("alt=")
            end_name = img_add[start_name+5:].find('"')
            image_name = img_add[start_name+5:start_name+end_name+5] + '.jpg'
            self.img.append(image_name)

#             Witing Image in Local Drive
            response = requests.get(image_url)
            with open(image_name, 'wb') as f:
                f.write(response.content)

#             Loocking for Image descriptions
            p = self.driver.find_elements(By.CSS_SELECTOR, "div.caption")
            for n,i in enumerate(p):
                self.txt.append(i.find_element(By.TAG_NAME, "p").text)

#             Save data in a dictionary    
            for i in range(len(self.img)):
                self.img_dict[self.img[i]] = self.txt[i]

    def save_as_json(self, file_path):
        with open(file_path, "w") as json_file:
            json.dump(self.img_dict, json_file)

# Usage
scraper = ImageScraper()
scraper.scrape_images("https://earthobservatory.nasa.gov/images", 5)
scraper.save_as_json("img_dict.json")